# Análise de Sentimentos

Este notebook faz o treinando de machine learning de um modelo de análise de sentimentos, utilizando o dataset do Buscapé fornecido (confidencial), de forma que:
- Carrega e estrutura os dados do dataset em dicionário
- Exibe estatísticas acerca da do dataset
- Apresenta comentários acerca dos resultados obtidos

## Configurações Iniciais

Primeiramente, fazer o ajuste no path do Python para a raíz do projeto:

In [1]:
import os
os.chdir('..')

--------------------------------------

## Processamento do Dataset

Carregar a bliblioteca de normalização de texto e de dump de dados:

In [2]:
from nlputils.lexical.normalizer import Normalizer
import pickle

Instanciar um objeto para fazer as normalizações necessárias:

In [3]:
normalizer = Normalizer()

Implementar uma rotina de pré-processamento a ser aplicada em todo o dataset antes de construir o modelo e uma para auxiliar o carregamento dos dados, evitando replicação de código.

A função ```preprocessing``` recebe um texto (string) de entrada e o deixa normalizado, conforme procedimentos de nível lexical, e retorna o próprio texto (string) mais compacto.

In [4]:
def preprocessing(text):
    text = normalizer.to_lowercase(text)
    text = normalizer.remove_ponctuations(text)
    tokens = normalizer.tokenize_words(text)
    tokens = normalizer.remove_stopwords(tokens)
    return ' '.join(tokens)

A função ```all_files``` recebe o caminho do dataset a ser carregado e gera o caminho de todos os arquivos XML a serem lidos. O caminho de entrada deve terminar com ```/```.

In [5]:
def all_files(dataset_path):
    for directory in os.listdir(dataset_path):
        if directory != '.DS_Store':
            for stars in os.listdir(dataset_path + directory):
                for file in os.listdir(dataset_path + directory + '/' + stars):
                    if file.endswith('.xml'):
                        yield '{}{}/{}/{}'.format(dataset_path, directory, stars, file)

A função ```dumpvar``` salva em dados binário todo o valor de uma variável na memória. Basta fornecer o ponteiro do identificador da variável e o caminho onde será salvo o dump.

In [6]:
def dumpvar(var, path):
    path_itens = path.split('/')
    path_dir = '/'.join(path_itens[:-1]) + '/'
    
    if not os.path.isdir(path_dir):
        os.makedirs(path_dir)
    
    with open(path, 'wb') as fp:
        pickle.dump(var, fp)

A função ```loadvar``` retorna o valor de uma dump de memória salvo em disco. Basta o caminho onde está salvo o dump e atribuir em uma variável.

In [7]:
def loadvar(path):
    with open(path, 'rb') as fp:
        return pickle.load(fp)

---------------------------------

## Carregando o Dataset

Importar as bibliotecas utilizadas para fazer o parse do XML e criar a estrutura.

In [8]:
import xmltodict
from xml.parsers.expat import ExpatError
import pandas as pd
import time

Carregando os arquivos XML, exibindo estatísticas sobre eles e criando a estrutura que será utilizada no modelo de Machine Learning.

In [9]:
dataset = {'polarity': [], 'pros': [], 'cons': [], 'review': []}
count_ok = 0
count_err = 0

start_time = time.perf_counter()

for file in all_files('data/trainset/'):
    filename = file.split('/')[-1]
    with open(file, 'r') as text_file:
        data = text_file.read()
        try:
            dict_data = xmltodict.parse(data)
            polarity = float(dict_data['review']['stars']['@value'])
            dataset['polarity'].append(int(polarity))
            dataset['review'].append(str(dict_data['review']['opinion']))
            dataset['pros'].append(str(dict_data['review']['pros']))
            dataset['cons'].append(str(dict_data['review']['cons']))
            count_ok += 1
        except ExpatError:
            print('Arquivo "{}" está com xml mal formatado'.format(file))
            count_err += 1

end_time = time.perf_counter()

print('-------------------------------------------')
print('TEMPO DE EXECUÇÃO: {:.2f} segundos'.format(end_time - start_time))
print('Arquivos carregados OK:', count_ok)
print('Arquivos com ERRO:', count_err)

Arquivo "data/trainset/Celular e Smartphone/4.0/2_42055.xml" está com xml mal formatado
Arquivo "data/trainset/Umidificador/4.0/7_393334.xml" está com xml mal formatado
Arquivo "data/trainset/Notebook/3.0/0_343109.xml" está com xml mal formatado
Arquivo "data/trainset/HD/5.0/0_336852.xml" está com xml mal formatado
Arquivo "data/trainset/Tablet/4.0/0_335616.xml" está com xml mal formatado
-------------------------------------------
TEMPO DE EXECUÇÃO: 17.16 segundos
Arquivos carregados OK: 67025
Arquivos com ERRO: 5


### Estatísticas acerca do dataset

Quantidade de comentários:

In [10]:
print(count_ok)

67025


Quantidade de sentenças:

In [11]:
pass

Quantidade de comentários por score:

In [12]:
for i in range(0, 6):
    print('Score #{}: {}'.format(i, dataset['polarity'].count(i)))

Score #0: 671
Score #1: 2396
Score #2: 2892
Score #3: 8936
Score #4: 25775
Score #5: 26355


---------------------------------

## T

Imporatar biblioteca para aleatorizar a escolha de dados entre treinamento e teste.

In [25]:
import random

Aqui, o dataset está sendo repartido para criar o conjunto de treinamento e o de teste. Aproximadamente 75% dos dados serão usados para treinamento e 25% para teste. Cada um escolhido entre teste e treino será dado de maneira aleatória.

In [27]:
size_test = count_ok // 4
size_train = count_ok - size_test

dataset['trainset'] = ['train'] * size_train + ['test'] * size_test

random.shuffle(dataset['trainset'])

In [28]:
dataframe = pd.DataFrame(data=dataset)
# dataframe.head()

Agora vamos ler todos os textos e adicionar um novo campo na dataframe que irá ter a sentença sem pontuações e stopwords

In [29]:
start_time = time.perf_counter()

dataframe['review'] = dataframe['review'].apply(preprocessing)
dataframe['pros'] = dataframe['pros'].apply(preprocessing)
dataframe['cons'] = dataframe['cons'].apply(preprocessing)

end_time = time.perf_counter()
print('TEMPO DE EXECUÇÃO: {:.2f} segundos'.format(end_time - start_time))

# dataframe.head()

TEMPO DE EXECUÇÃO: 57.52 segundos


In [18]:
dumpvar(dataframe, 'data/dump/dataframe.pickle')

In [ ]:
# dataframe = loadvar('data/dump/dataframe.pickle')

----------------------------

## T

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [31]:
train_reviews = dataframe[dataframe['trainset'] == 'train']['review'].values.tolist()
train_classes = dataframe[dataframe['trainset'] == 'train']['polarity'].values.tolist()
test_reviews = dataframe[dataframe['trainset'] == 'test']['review'].values.tolist()
test_classes = dataframe[dataframe['trainset'] == 'test']['polarity'].values.tolist()

transformer = TfidfVectorizer()
transformer.fit(train_reviews)
X = transformer.transform(train_reviews)
X_test = transformer.transform(test_reviews)

Treinando 

In [40]:
# classifier = LogisticRegression(solver='warn', multi_class='warn')
classifier = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=200, n_jobs=4)

start_time = time.perf_counter()
classifier.fit(X, train_classes)
end_time = time.perf_counter()
print('TEMPO DE EXECUÇÃO: {:.2f} segundos'.format(end_time - start_time))

TEMPO DE EXECUÇÃO: 39.89 segundos


In [41]:
accuracy_score(test_classes, classifier.predict(X_test))

0.5625447600859393

In [24]:
dumpvar(classifier, 'data/dump/classifier.pickle')

In [ ]:
# classifier = loadvar('data/dump/classifier.pickle')